In [1]:
setwd('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1012. NC Well Arsenic/P1012.2. Analyses/P1012.2.2. Arsenic Prediction')
Output = ('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1012. NC Well Arsenic/P1012.2. Analyses/P1012.2.2. Arsenic Prediction/Output')
cur_date = "081423"

library(readxl)
library(openxlsx)
library(lubridate)
library(tidyverse)
library(gtsummary)
library(caret)
library(e1071)
library(Hmisc)
library(randomForest)
library(pROC)
library(themis)
#library(xgboost)
library(rlang)

# reading in file
well_data = data.frame(read_excel("Input/Imputed_Well_Data_080823.xlsx")) 


Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr   1.1.2     ✔ readr   2.1.4
✔ forcats 1.0.0     ✔ stringr 1.5.0
✔ ggplot2 3.4.2     ✔ tibble  3.2.1
✔ purrr   1.0.1     ✔ tidyr   1.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Loading required package: lattice


Attaching package: ‘caret’


The following object is masked from ‘package:purrr’:

    lift



Attaching package: ‘Hmisc’


The following object is masked from ‘package:e1071’:

    impute


The following objects are masked from ‘package:dplyr’:

    src, summarize


The following objects are masked from ‘package:base’:

    format.pval, units


rand

In [2]:
head(well_data)

,Tax_ID,Health_Dept_ID,Permit_No,Water_Sample_Date,Casing_Depth,Well_Depth,Static_Water_Depth,Flow_Rate,pH,Geology,Soil_Type,Landuse,Longtitude,Latitude,Stream_Distance,Elevation,Metal,Concentration,Detect_Concentration
,<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>
1,1006004,960,12-155,9/24/12,52,165,41,60.0,7.7,Carolina Slate Belt - CZmd3,BdB2,81,-80.29918,35.17970,811.9613,611.499,Ar,1.531414,ND
2,1024009,1803,15-177,12/17/15,40,445,42,2.0,7.3,Carolina Slate Belt - CZfv2,GsB,81,-80.31061,35.15487,341.7654,688.452,Ar,2.348528,ND
3,1054019,1647,14-212,2/2/15,45,160,40,40.0,7.4,Carolina Slate Belt - CZmd3,GsB,81,-80.33170,35.16158,634.2669,692.789,Ar,2.043848,ND
4,1057017,970,12-169,10/22/12,42,440,57,1.5,8.0,Carolina Slate Belt - CZmd2,BaB,42,-80.32263,35.13962,855.4713,692.789,Ar,35.000000,D
5,1060006,533,10-239,1/3/11,48,120,42,25.0,7.1,Carolina Slate Belt - CZmd2,BdB2,43,-80.32911,35.13027,163.5688,683.387,Ar,2.297118,ND
6,1066006,1795,15-224,12/15/15,60,280,32,10.0,8.2,Carolina Slate Belt - CZmd2,BaB,22,-80.32205,35.10479,725.7672,614.599,Ar,53.000000,D


Using `water_sample_date`, `casing_depth`, `well_depth`, `static_water_depth`, `pH`, `flow_rate`, `geology`, `soil_type`, `latitude`, and `longtitude` to predict with As concentration falls above or below a certain threshold. RF and models will be built to predict concentration that has binarized in 3 different ways:

1. Detect vs. Non-Detect
2. < or > 5ppb
3. < or > 10ppb

Starting by creating 2 additional variables for above and below 5 and 10ppb and calculating some summary statistics to determine if there are any signficiant differenes between the predictor variables for each outcome variable, but omitted categorical variables for simplicity. 

WILL NEED TO GO BACK AND CONDENSE SOME OF THE GEOLOGY AND SOIL TYPE CATEGORIES.

In [3]:
arsenic_data = well_data %>%
    mutate(Five_Threshold = relevel(factor(ifelse(Concentration >= 5, 1, 0)), ref = "0"),
          Ten_Threshold = relevel(factor(ifelse(Concentration >= 10, 1, 0)), ref = "0"),
           # making this col into a factor
          Detect_Concentration = relevel(factor(ifelse(Detect_Concentration == "D", 1, 0)), ref = "0"),
          # converting water sample date from a character to a date type 
          Water_Sample_Date = mdy(Water_Sample_Date)) %>%

    # filtering for Arsenic only
    filter(Metal == "Ar") %>%
    # condensing geological categories 
    separate(Geology, c(NA, "Geology"), sep = "-") %>%
    # removing landuse since it won't be a predictor
    select(-Landuse) 

head(arsenic_data)

Warning message:
“There was 1 warning in `mutate()`.
ℹ In argument: `Water_Sample_Date = mdy(Water_Sample_Date)`.
Caused by warning:
!  6 failed to parse.”


,Tax_ID,Health_Dept_ID,Permit_No,Water_Sample_Date,Casing_Depth,Well_Depth,Static_Water_Depth,Flow_Rate,pH,Geology,Soil_Type,Longtitude,Latitude,Stream_Distance,Elevation,Metal,Concentration,Detect_Concentration,Five_Threshold,Ten_Threshold
,<chr>,<dbl>,<chr>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<fct>,<fct>,<fct>
1,1006004,960,12-155,2012-09-24,52,165,41,60.0,7.7,CZmd3,BdB2,-80.29918,35.17970,811.9613,611.499,Ar,1.531414,0,0,0
2,1024009,1803,15-177,2015-12-17,40,445,42,2.0,7.3,CZfv2,GsB,-80.31061,35.15487,341.7654,688.452,Ar,2.348528,0,0,0
3,1054019,1647,14-212,2015-02-02,45,160,40,40.0,7.4,CZmd3,GsB,-80.33170,35.16158,634.2669,692.789,Ar,2.043848,0,0,0
4,1057017,970,12-169,2012-10-22,42,440,57,1.5,8.0,CZmd2,BaB,-80.32263,35.13962,855.4713,692.789,Ar,35.000000,1,1,1
5,1060006,533,10-239,2011-01-03,48,120,42,25.0,7.1,CZmd2,BdB2,-80.32911,35.13027,163.5688,683.387,Ar,2.297118,0,0,0
6,1066006,1795,15-224,2015-12-15,60,280,32,10.0,8.2,CZmd2,BaB,-80.32205,35.10479,725.7672,614.599,Ar,53.000000,1,1,1


In [4]:
# original number of records
dim(well_data)

# records kept for analysis
dim(arsenic_data)

[1] 2145   19

[1] 715  20

In [5]:
# summary statistics df: detect_concentration
arsenic_data %>%
  tbl_summary(by = Detect_Concentration, missing = "no", 
  include = colnames(arsenic_data[c(4:15)]), 
              statistic = list(all_continuous() ~ "{mean} ({sd})")) %>%
  add_n() %>% 
  add_p(test = list(all_continuous() ~ "aov")) %>% # adding p value from anova
  as_tibble()

There was an error in 'add_p()/add_difference()' for variable 'Geology', p-value omitted:
Error in stats::fisher.test(c(" CZmd3", " CZfv2", " CZmd3", " CZmd2", : FEXACT error 7(location). LDSTP=18510 is too small for this problem,
  (pastp=33.69, ipn_0:=ipoin[itp=440]=17951, stp[ipn_0]=17.2264).
Increase workspace or consider using 'simulate.p.value=TRUE'

There was an error in 'add_p()/add_difference()' for variable 'Soil_Type', p-value omitted:
Error in stats::fisher.test(c("BdB2", "GsB", "GsB", "BaB", "BdB2", "BaB", : FEXACT error 7(location). LDSTP=18390 is too small for this problem,
  (pastp=11.3975, ipn_0:=ipoin[itp=528]=12000, stp[ipn_0]=6.49224).
Increase workspace or consider using 'simulate.p.value=TRUE'



**Characteristic**,**N**,"**0**, N = 517","**1**, N = 198",**p-value**
<chr>,<chr>,<chr>,<chr>,<chr>
Water_Sample_Date,713,2013-06-05 (979.174260670888),2013-03-05 (957.843005291701),0.3
Casing_Depth,715,74 (33),55 (23),<0.001
Well_Depth,715,301 (144),334 (128),0.005
Static_Water_Depth,715,35 (12),36 (13),0.5
Flow_Rate,715,25 (33),14 (16),<0.001
pH,715,7.45 (0.54),7.82 (0.40),<0.001
Geology,715,NA,NA,NA
CZfv,NA,118 (23%),7 (3.5%),NA
CZfv2,NA,7 (1.4%),2 (1.0%),NA


In [6]:
# summary statistics df: five_threshold
arsenic_data %>%
  tbl_summary(by = Five_Threshold, missing = "no", 
  include = colnames(arsenic_data[c(4:15)]), 
              statistic = list(all_continuous() ~ "{mean} ({sd})")) %>%
  add_n() %>% 
  add_p(test = list(all_continuous() ~ "aov")) %>% # adding p value from anova
  as_tibble()

There was an error in 'add_p()/add_difference()' for variable 'Geology', p-value omitted:
Error in stats::fisher.test(c(" CZmd3", " CZfv2", " CZmd3", " CZmd2", : FEXACT error 7(location). LDSTP=18510 is too small for this problem,
  (pastp=37.197, ipn_0:=ipoin[itp=363]=18102, stp[ipn_0]=34.4626).
Increase workspace or consider using 'simulate.p.value=TRUE'

There was an error in 'add_p()/add_difference()' for variable 'Soil_Type', p-value omitted:
Error in stats::fisher.test(c("BdB2", "GsB", "GsB", "BaB", "BdB2", "BaB", : FEXACT error 7(location). LDSTP=18390 is too small for this problem,
  (pastp=16.9551, ipn_0:=ipoin[itp=126]=3667, stp[ipn_0]=1.38629).
Increase workspace or consider using 'simulate.p.value=TRUE'



**Characteristic**,**N**,"**0**, N = 521","**1**, N = 194",**p-value**
<chr>,<chr>,<chr>,<chr>,<chr>
Water_Sample_Date,713,2013-06-01 (979.73581125107),2013-03-12 (956.560244097347),0.3
Casing_Depth,715,74 (33),55 (23),<0.001
Well_Depth,715,300 (144),336 (129),0.003
Static_Water_Depth,715,35 (12),35 (13),0.6
Flow_Rate,715,25 (33),14 (16),<0.001
pH,715,7.45 (0.54),7.83 (0.40),<0.001
Geology,715,NA,NA,NA
CZfv,NA,118 (23%),7 (3.6%),NA
CZfv2,NA,7 (1.3%),2 (1.0%),NA


In [7]:
# summary statistics df: ten_threshold
arsenic_data %>%
  tbl_summary(by = Ten_Threshold, missing = "no", 
  include = colnames(arsenic_data[c(4:15)]), 
              statistic = list(all_continuous() ~ "{mean} ({sd})")) %>%
  add_n() %>% 
  add_p(test = list(all_continuous() ~ "aov")) %>% # adding p value from anova
  as_tibble()

There was an error in 'add_p()/add_difference()' for variable 'Geology', p-value omitted:
Error in stats::fisher.test(c(" CZmd3", " CZfv2", " CZmd3", " CZmd2", : FEXACT error 7(location). LDSTP=18510 is too small for this problem,
  (pastp=32.2004, ipn_0:=ipoin[itp=525]=18099, stp[ipn_0]=20.263).
Increase workspace or consider using 'simulate.p.value=TRUE'

There was an error in 'add_p()/add_difference()' for variable 'Soil_Type', p-value omitted:
Error in stats::fisher.test(c("BdB2", "GsB", "GsB", "BaB", "BdB2", "BaB", : FEXACT error 7(location). LDSTP=18390 is too small for this problem,
  (pastp=9.36426, ipn_0:=ipoin[itp=91]=4350, stp[ipn_0]=7.59085).
Increase workspace or consider using 'simulate.p.value=TRUE'



**Characteristic**,**N**,"**0**, N = 583","**1**, N = 132",**p-value**
<chr>,<chr>,<chr>,<chr>,<chr>
Water_Sample_Date,713,2013-05-12 (972.978329886697),2013-05-03 (979.450720926567),>0.9
Casing_Depth,715,73 (33),51 (19),<0.001
Well_Depth,715,306 (147),327 (109),0.12
Static_Water_Depth,715,35 (13),34 (11),0.4
Flow_Rate,715,24 (32),13 (15),<0.001
pH,715,7.48 (0.54),7.87 (0.41),<0.001
Geology,715,NA,NA,NA
CZfv,NA,123 (21%),2 (1.5%),NA
CZfv2,NA,8 (1.4%),1 (0.8%),NA


Although the p values are very significant between the classes of each outcome, there is a high level of class imbalance which is likely to affect model performance. Class imbalance will be addressed using SMOTE. 

In [8]:
# creating dfs for each outcome
# dropped 2 rows that had missing dates
arsenic_detect_df = drop_na(arsenic_data[,c(4:15,18)]) 
arsenic_five_df = drop_na(arsenic_data[,c(4:15,19)]) 
arsenic_ten_df = drop_na(arsenic_data[,c(4:15,20)]) 

head(arsenic_detect_df)

,Water_Sample_Date,Casing_Depth,Well_Depth,Static_Water_Depth,Flow_Rate,pH,Geology,Soil_Type,Longtitude,Latitude,Stream_Distance,Elevation,Detect_Concentration
,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,2012-09-24,52,165,41,60.0,7.7,CZmd3,BdB2,-80.29918,35.17970,811.9613,611.499,0
2,2015-12-17,40,445,42,2.0,7.3,CZfv2,GsB,-80.31061,35.15487,341.7654,688.452,0
3,2015-02-02,45,160,40,40.0,7.4,CZmd3,GsB,-80.33170,35.16158,634.2669,692.789,0
4,2012-10-22,42,440,57,1.5,8.0,CZmd2,BaB,-80.32263,35.13962,855.4713,692.789,1
5,2011-01-03,48,120,42,25.0,7.1,CZmd2,BdB2,-80.32911,35.13027,163.5688,683.387,0
6,2015-12-15,60,280,32,10.0,8.2,CZmd2,BaB,-80.32205,35.10479,725.7672,614.599,1


# Random Forest
- an ensemble learning method operating by constructing a multitude of decision trees at training time, which uses multiple methods to obtain a better predictive performance and includes bagging and random forest
- algorithm uses a bootstrop aggregation, to reduce overfitting the training datset but only a subset of the features are used hence decorrelation of predictors

In [9]:
rf_classification = function(dataset, outcome, pred_outcome){
    # setting for reproducibility
    set.seed(12)
    # splitting data into training and testing sets
    dataset_index = createFolds(dataset[[outcome]], k = 5) # 5 fold CV
    
    ntree_values = c(50, 250, 500) # number of trees 
    p = dim(dataset)[2] - 1 # number of variables in dataset
    mtry_values = c(sqrt(p), p/2, p/3) # number of predictors

    metrics = data.frame()
    variable_importance_df = data.frame()
    roc_objects = c()
    threshold_data = data.frame()
    
    for (i in 1:length(dataset_index)){
        
        data_train = dataset[-dataset_index[[i]],]
        # using SMOTE to balance the class distribution
        balanced_data_train = smotenc(data_train, outcome)
        data_test = dataset[dataset_index[[i]],]

        # will use ntree and mtry values to determine which combination yields the smallest MSE
        reg_rf_pred_tune = list()
        rf_OOB_errors = list()
        rf_error_df = data.frame()
        for (j in 1:length(ntree_values)){
            for (k in 1:length(mtry_values)){
                reg_rf_pred_tune[[k]] = randomForest(as.formula(paste0(outcome, "~.")), data = balanced_data_train, 
                                                     ntree = ntree_values[j], mtry = mtry_values[k])
                rf_OOB_errors[[k]] = data.frame("Tree Number" = ntree_values[j], "Variable Number" = mtry_values[k], 
                                       "OOB_errors" = reg_rf_pred_tune[[k]]$err.rate[ntree_values[j],1])
                rf_error_df = rbind(rf_error_df, rf_OOB_errors[[k]])
            }
        }

        # finding the lowest OOB error using best number of predictors at split and refitting OG tree
        best_oob_errors <- which(rf_error_df$OOB_errors == min(rf_error_df$OOB_errors))

        reg_rf <- randomForest(as.formula(paste0(outcome, "~.")), data = balanced_data_train,
                               ntree = rf_error_df$Tree.Number[min(best_oob_errors)],
                               mtry = rf_error_df$Variable.Number[min(best_oob_errors)])

        # predicting on test set
        data_test[[pred_outcome]] = predict(reg_rf, newdata = data_test, type = "response")
        
        matrix = confusionMatrix(data = data_test[[pred_outcome]], reference = data_test[[outcome]], 
                                     positive = "1")

        # calculating AUC
        auc = auc(response = data_test[[outcome]], predictor = factor(data_test[[pred_outcome]], ordered = TRUE))

        
        # calculating values to plot ROC curve later
        roc_obj = roc(response = data_test[[outcome]], predictor = factor(data_test[[pred_outcome]], ordered = TRUE))

        # Return max Youden's index, with specificity and sensitivity
        best_thres_data = data.frame(coords(roc_obj, x = "best", best.method = c("youden", "closest.topleft")))
        threshold_data = rbind(threshold_data, best_thres_data)
        
        # extracting accuracy, sens, spec, PPV to take mean later
        matrix_values = data.frame(t(c(matrix$byClass[11])), t(c(matrix$byClass[1:3])), auc)
        
        # extracting variable importance
        var_importance_values = data.frame(importance(reg_rf)) %>%
            rownames_to_column(var = "Predictor")
        variable_importance_df = rbind(variable_importance_df, var_importance_values)
   
        # adding values to df
        metrics = rbind(metrics, matrix_values)
        
    }
    
    # taking averages/sd 
    metrics = metrics %>%
        summarise(`Balanced Accuracy` = mean(Balanced.Accuracy), Sensitivity = mean(Sensitivity), 
              Specificity = mean(Specificity), PPV = mean(Pos.Pred.Value), AUC = mean(auc))
    
    variable_importance_df = variable_importance_df %>%
        group_by(Predictor) %>%
        summarise(MeanDecreaseGini = mean(MeanDecreaseGini)) %>%
        # sorting by most important variables
        arrange(-MeanDecreaseGini)
  
    # return training set, matrix, variable importance values, (last) roc object, best threshold data
    return(list(balanced_data_train, metrics, variable_importance_df, roc_obj, threshold_data))

}

In [10]:
# calling fn
rf_values_arsenic_detect = rf_classification(arsenic_detect_df, "Detect_Concentration", 
                                             "pred_Detect_Concentration")
rf_values_arsenic_five = rf_classification(arsenic_five_df, "Five_Threshold", 
                                             "pred_Five_Threshold")
rf_values_arsenic_ten = rf_classification(arsenic_ten_df, "Ten_Threshold", 
                                             "pred_Ten_Threshold")

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting leve

In [11]:
# viewing results
rf_confusion_matrix = data.frame(Model = rep(c("RF Classification"), times = 3), Kernel = NA,
    Outcome = c("Detect_Concentration", "Five_Threshold", "Ten_Threshold"),
            rbind(rf_values_arsenic_detect[[2]], rf_values_arsenic_five[[2]], rf_values_arsenic_ten[[2]]))

rf_confusion_matrix

# viewing most significant features
rf_values_arsenic_detect[[3]]
rf_values_arsenic_five[[3]]
rf_values_arsenic_ten[[3]]

Model,Kernel,Outcome,Balanced.Accuracy,Sensitivity,Specificity,PPV,AUC
<chr>,<lgl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
RF Classification,NA,Detect_Concentration,0.7357313,0.5860256,0.8854369,0.6695726,0.7357313
RF Classification,NA,Five_Threshold,0.7208617,0.5572200,0.8845034,0.6482131,0.7208617
RF Classification,NA,Ten_Threshold,0.6656278,0.4000000,0.9312555,0.5837460,0.6656278


Predictor,MeanDecreaseGini
<chr>,<dbl>
Longtitude,105.93236
pH,46.70859
Latitude,36.68045
Casing_Depth,36.31757
Geology,34.21661
Well_Depth,28.44958
Flow_Rate,27.47713
Elevation,21.99355
Water_Sample_Date,20.65661


Predictor,MeanDecreaseGini
<chr>,<dbl>
Longtitude,105.73680
pH,47.61830
Latitude,37.54380
Geology,37.27578
Casing_Depth,37.01189
Well_Depth,28.53321
Flow_Rate,27.09315
Elevation,20.46601
Water_Sample_Date,19.83986


Predictor,MeanDecreaseGini
<chr>,<dbl>
Longtitude,140.64472
Geology,50.11961
pH,49.78990
Casing_Depth,44.77534
Latitude,42.63574
Flow_Rate,26.53687
Well_Depth,23.90546
Static_Water_Depth,21.63356
Elevation,18.62940


RF didn't perform as well as I hoped, so let's try another model - SVM. 

# Support Vector Machine (SVM)
- predicts by projecting predictors onto a high dimensional space and uses kernels to make the data more separable (unfortunately makes interpretability of model results more difficult)
- does a better job at handling a large number of predictors since p > n

In [16]:
# model errors are calculated to assess the predictive accuracy of the model
svm_classification = function(dataset, model, outcome, pred_outcome, gamma_values, cost_values, elsilon_values){
  
    # setting seed for reproducibility
    set.seed(12)
    
    # splitting data into training and testing sets
    dataset_index = createFolds(dataset[[outcome]], k = 5) #5 fold CV
    metrics = data.frame()
    for (i in 1:length(dataset_index)){
        data_train = dataset[-dataset_index[[i]],]
        # using SMOTE to balance the class distribution
        balanced_data_train = smotenc(data_train, outcome)
        
        data_test = dataset[dataset_index[[i]],]
        print(colnames(balanced_data_train))
        print(colnames(data_test))

        # now pruning parameters (based on the training dataset to prevent overfitting)
        svr_tune <- tune(svm, as.formula(paste0(outcome, "~.")), data = balanced_data_train, kernel = model, 
                       ranges = list(elsilon = elsilon_values, cost = cost_values, gamma = gamma_values)) 

        # choosing best model
        best_svm <- svr_tune$best.model

        # predicting with tuned parameters 
        data_test[[pred_outcome]] <- predict(best_svm, newdata = data_test, type = "response")

        matrix = confusionMatrix(data = data_test[[pred_outcome]], reference = data_test[[outcome]], 
                                 positive = "1")
 
        # calculating AUC
        auc = auc(response = data_test[[outcome]], predictor = factor(data_test[[pred_outcome]], ordered = TRUE))
        # extracting accuracy, sens, spec, PPV to take mean later
        matrix_values = data.frame(t(c(matrix$byClass[11])), t(c(matrix$byClass[1:3])), auc)
   
        # adding values to df
        metrics = rbind(metrics, matrix_values)
  }
  
  # taking averages/sd 
  metrics = metrics %>%
    summarise(`Balanced Accuracy` = mean(Balanced.Accuracy), Sensitivity = mean(Sensitivity), 
              Specificity = mean(Specificity), PPV = mean(Pos.Pred.Value), AUC = mean(auc))
  
  return(metrics)
}

In [17]:
# calling fn
# linear
linear_svm_values_arsenic_detect = svm_classification(arsenic_detect_df, "linear", "Detect_Concentration", 
                                             "pred_Detect_Concentration", 0.035, 1:5, seq(0,1,0.2))
linear_svm_values_arsenic_five = svm_classification(arsenic_five_df, "linear", "Five_Threshold", 
                                             "pred_Five_Threshold", 0.035, 1:5, seq(0,1,0.2))
linear_svm_values_arsenic_ten = svm_classification(arsenic_ten_df, "linear", "Ten_Threshold", 
                                             "pred_Ten_Threshold", 0.035, 1:5, seq(0,1,0.2))

# radial
radial_svm_values_arsenic_detect = svm_classification(arsenic_detect_df, "radial", "Detect_Concentration", 
                                             "pred_Detect_Concentration", 0.035, 1:5, seq(0,1,0.2))
radial_svm_values_arsenic_five = svm_classification(arsenic_five_df, "radial", "Five_Threshold", 
                                             "pred_Five_Threshold", 0.035, 1:5, seq(0,1,0.2))
radial_svm_values_arsenic_ten = svm_classification(arsenic_ten_df, "radial", "Ten_Threshold", 
                                             "pred_Ten_Threshold", 0.035, 1:5, seq(0,1,0.2))

# polynomial
polynomial_svm_values_arsenic_detect = svm_classification(arsenic_detect_df, "polynomial", "Detect_Concentration", 
                                             "pred_Detect_Concentration", 0.035, 1:5, 0.1)
polynomial_svm_values_arsenic_five = svm_classification(arsenic_five_df, "polynomial", "Five_Threshold", 
                                             "pred_Five_Threshold", 0.035, 1:5, 0.1)
polynomial_svm_values_arsenic_ten = svm_classification(arsenic_ten_df, "polynomial", "Ten_Threshold", 
                                             "pred_Ten_Threshold", 0.035, 1:5, 0.1)

 [1] "Water_Sample_Date"    "Casing_Depth"         "Well_Depth"          
 [4] "Static_Water_Depth"   "Flow_Rate"            "pH"                  
 [7] "Geology"              "Soil_Type"            "Longtitude"          
[10] "Latitude"             "Stream_Distance"      "Elevation"           
[13] "Detect_Concentration"
 [1] "Water_Sample_Date"    "Casing_Depth"         "Well_Depth"          
 [4] "Static_Water_Depth"   "Flow_Rate"            "pH"                  
 [7] "Geology"              "Soil_Type"            "Longtitude"          
[10] "Latitude"             "Stream_Distance"      "Elevation"           
[13] "Detect_Concentration"


ERROR: Error in predict.svm(model, if (!is.null(validation.x)) validation.x else if (useFormula) data[-train.ind[[sample]], : test data does not match model !


In [13]:
# viewing results
svm_confusion_matrix = data.frame(Model = "SVM Classification",
        Outcome = c("Detect_Concentration", "Five_Threshold", "Ten_Threshold"),
        Kernel = rep(c(rep(c("linear"), times = 3), rep(c("radial"), times = 3), rep(c("polynomial"), times = 3))),
            rbind(linear_svm_values_arsenic_detect, linear_svm_values_arsenic_five, linear_svm_values_arsenic_ten, 
            radial_svm_values_arsenic_detect, radial_svm_values_arsenic_five, radial_svm_values_arsenic_ten, 
            polynomial_svm_values_arsenic_detect, polynomial_svm_values_arsenic_five, 
            polynomial_svm_values_arsenic_ten))

svm_confusion_matrix

Model,Outcome,Kernel,Balanced.Accuracy,Sensitivity,Specificity,PPV,AUC
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
SVM Classification,Detect_Concentration,linear,0.7191856,0.7914085,0.6469627,0.4731488,0.7191856
SVM Classification,Five_Threshold,linear,0.7255411,0.7954023,0.6556798,0.4765951,0.7255411
SVM Classification,Ten_Threshold,linear,0.7310138,0.8136585,0.6483690,0.3683656,0.7310138
SVM Classification,Detect_Concentration,radial,0.7127112,0.7811806,0.6442419,0.4685575,0.7127112
SVM Classification,Five_Threshold,radial,0.7151175,0.7813672,0.6488678,0.4672122,0.7151175
SVM Classification,Ten_Threshold,radial,0.7255548,0.7696341,0.6814754,0.3806995,0.7255548
SVM Classification,Detect_Concentration,polynomial,0.6581778,0.9317358,0.3846197,0.3775792,0.6581778
SVM Classification,Five_Threshold,polynomial,0.6528688,0.9096794,0.3960581,0.3730571,0.6528688
SVM Classification,Ten_Threshold,polynomial,0.6483469,0.9069512,0.3897426,0.2716422,0.6483469


## SVM - Variable Importance
Unfortunately, I was unable to find a function that extracts variable importance as with random forest. However, variable importance is simply the difference in overall predictivity on a model in that variable's absence in the model. Therefore, SVM models will be rerun without one variables each time. Additionally, negative value for variable importance for SVM means that inclusion of that predictor in models hurt predictivity while a positive number means that inclusion of that predictor improved predictivity. Since linear SVM models performed the best only those variable importance values will be extracted.

Getting variable importance for ....

In [14]:
# model errors are calculated to assess the predictive accuracy of the model
svm_var_imp = function(dataset, model, outcome, pred_outcome, gamma_values, cost_values, elsilon_values){
  
    # setting seed for reproducibility
    set.seed(12)
    
    metrics = data.frame()
    # iterating through variables in the dataset
    predictors = unique(colnames(dataset[-7]))
    for (i in 1:length(predictors)){
        subsetted_df = dataset[,colnames(dataset[-i])]
        
        # splitting data into training and testing sets
        dataset_index = createFolds(subsetted_df[[outcome]], k = 5) #5 fold CV
        for (j in 1:length(dataset_index)){
            data_train = subsetted_df[-dataset_index[[j]],]
            # using SMOTE to balance the class distribution
            balanced_data_train = smotenc(data_train, outcome)

            data_test = subsetted_df[dataset_index[[j]],]

            # now pruning parameters (based on the training dataset to prevent overfitting)
            svr_tune <- tune(svm, as.formula(paste0(outcome, "~.")), data = balanced_data_train, kernel = model, 
                           ranges = list(elsilon = elsilon_values, cost = cost_values, gamma = gamma_values)) 

            # choosing best model
            best_svm <- svr_tune$best.model

            # predicting with tuned parameters 
            data_test[[pred_outcome]] <- predict(best_svm, newdata = data_test, type = "response")

            matrix = confusionMatrix(data = data_test[[pred_outcome]], reference = data_test[[outcome]], 
                                     positive = "1")

            # extracting accuracy to take mean later
            matrix_values = data.frame(t(c(matrix$byClass[11])), predictors[i])

            # adding values to df
            metrics = rbind(metrics, matrix_values)
        }
     }
    
    colnames(metrics) = c("Balanced Accuracy", "Predictor")

    # taking averages/sd 
     metrics = metrics %>%
        group_by(Predictor) %>%
        summarise(`Balanced Accuracy` = mean(`Balanced Accuracy`)) %>%
        mutate(Outcome = outcome)

    return(metrics)
}

In [15]:
# calling fn
# linear
linear_svm_var_imp_arsenic_detect = svm_var_imp(arsenic_detect_df, "linear", "Detect_Concentration", 
                                             "pred_Detect_Concentration", 0.035, 1:5, seq(0,1,0.2))
linear_svm_var_imp_arsenic_five = svm_var_imp(arsenic_five_df, "linear", "Five_Threshold", 
                                             "pred_Five_Threshold", 0.035, 1:5, seq(0,1,0.2))
linear_svm_var_imp_arsenic_ten = svm_var_imp(arsenic_ten_df, "linear", "Ten_Threshold", 
                                             "pred_Ten_Threshold", 0.035, 1:5, seq(0,1,0.2))

# radial
radial_svm_var_imp_arsenic_detect = svm_var_imp(arsenic_detect_df, "radial", "Detect_Concentration", 
                                             "pred_Detect_Concentration", 0.035, 1:5, seq(0,1,0.2))
radial_svm_var_imp_arsenic_five = svm_var_imp(arsenic_five_df, "radial", "Five_Threshold", 
                                             "pred_Five_Threshold", 0.035, 1:5, seq(0,1,0.2))
radial_svm_var_imp_arsenic_ten = svm_var_imp(arsenic_ten_df, "radial", "Ten_Threshold", 
                                             "pred_Ten_Threshold", 0.035, 1:5, seq(0,1,0.2))

In [16]:
# creating 1 var imp df
#svm
svm_var_imp_df = rbind(linear_svm_var_imp_arsenic_detect, linear_svm_var_imp_arsenic_five, 
                    linear_svm_var_imp_arsenic_ten, radial_svm_var_imp_arsenic_detect, 
                       radial_svm_var_imp_arsenic_five, radial_svm_var_imp_arsenic_ten) %>%
    # var importance value for svm = balanced accuracy with a predictor - balanced accuracy w/o a predictor
    mutate(Importance = svm_confusion_matrix$Balanced.Accuracy - `Balanced Accuracy`, 
    # adding a col for the model
           Model = c(rep(c("Linear-SVM"), times = 18), rep(c("Radial-SVM"), times = 18))) %>% 
    select(-`Balanced Accuracy`)
#rf
rf_var_imp_df = data.frame(rbind(rf_values_arsenic_detect[[3]], rf_values_arsenic_five[[3]], 
                                 rf_values_arsenic_ten[[3]]),
                Outcome = c(rep(c("Detect_Concentration"), times = 6), rep(c("Five_Threshold"), times = 6), 
                           rep(c("Ten_Threshold"), times = 6))) %>%
    # renaming mean decrease gini to match the importance value for SVM
    rename(Importance = MeanDecreaseGini) %>%
    # adding a col for the model
    mutate(Model = "RF")

var_imp_df = rbind(svm_var_imp_df, rf_var_imp_df)

head(var_imp_df)

Predictor,Outcome,Importance,Model
<chr>,<chr>,<dbl>,<chr>
Casing_Depth,Detect_Concentration,0.002581443,Linear-SVM
Flow_Rate,Detect_Concentration,0.023128300,Linear-SVM
Static_Water_Depth,Detect_Concentration,0.013838678,Linear-SVM
Water_Sample_Numeric,Detect_Concentration,-0.003708077,Linear-SVM
Well_Depth,Detect_Concentration,-0.015815635,Linear-SVM
pH,Detect_Concentration,0.053139051,Linear-SVM


In [24]:
# combining in 1 df
final_df = full_join(full_join(rf_confusion_matrix, svm_confusion_matrix)) %>%
    arrange(Outcome, -Balanced.Accuracy)

final_df

Joining with `by = join_by(Model, Kernel, Outcome, Balanced.Accuracy,
Sensitivity, Specificity, PPV, AUC)`
Joining with `by = join_by(Model, Kernel, Outcome, Balanced.Accuracy,
Sensitivity, Specificity, PPV, AUC)`


Model,Kernel,Outcome,Balanced.Accuracy,Sensitivity,Specificity,PPV,AUC
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
SVM Classification,linear,Detect_Concentration,0.7191856,0.7914085,0.6469627,0.4731488,0.7191856
RF Classification,NA,Detect_Concentration,0.7177265,0.6030392,0.8324138,0.5911626,0.7177265
SVM Classification,radial,Detect_Concentration,0.7127112,0.7811806,0.6442419,0.4685575,0.7127112
XGBoost Classification,NA,Detect_Concentration,0.6995005,0.6311124,0.7678885,0.5226340,0.6995005
SVM Classification,polynomial,Detect_Concentration,0.6581778,0.9317358,0.3846197,0.3775792,0.6581778
SVM Classification,linear,Five_Threshold,0.7255411,0.7954023,0.6556798,0.4765951,0.7255411
SVM Classification,radial,Five_Threshold,0.7151175,0.7813672,0.6488678,0.4672122,0.7151175
RF Classification,NA,Five_Threshold,0.7039734,0.5869328,0.8210139,0.5670558,0.7039734
XGBoost Classification,NA,Five_Threshold,0.6945144,0.6135310,0.7754978,0.5191626,0.6764857


In [25]:
# exporting
write.xlsx(final_df, paste0(Output,"/", "As_Prediction_Confusion_Matrix_", cur_date, ".xlsx"), rowNames = FALSE)
write.xlsx(var_imp_df, paste0(Output,"/", "Variable_Importance_", cur_date, ".xlsx"), rowNames = FALSE)